# Plotting Your Data
----
To start out, you'll want to load your data from a file using numpy, and then put it into a dataframe using Pandas.  This makes the data much easier to work with as we'll need to select sub-sets of it frequently.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib notebook
data_filename = "PetScan_Sep14_Time_19_17_12.csv"

In [ ]:
d = np.loadtxt(data_filename, delimiter=',',dtype = np.int32)
data = pd.DataFrame(d, columns = ['x','theta','t','ch1','ch2','counts'])
data['rate'] = data['counts']/data['t']
display(data)
angles = data['theta'].unique()
print(angles)

This should show several rows of a table, and the ``unique()`` function will select out the values of the individual angles that are in the dataset.

We'll use this to make a set of plots for each angle.  It'll be messy at first, but it gives a brief bit of insight into what you've got.

In [ ]:
fig,ax = plt.subplots()
for a in angles:
    x_val = data[data['theta']==a]['x']
    rate_val = data[data['theta']==a]['rate']
    ax.plot(x_val,rate_val,label=a,marker='.')
ax.legend()

Now, there's the obvious problem that we want ultimately want to work with spatial variables, not arbitrary position values.  To this end, you'll need to identify what value corresponds to the platform being centered between the detectors and change the ``center`` variable to reflect that.
You'll also want to convert your distances to units of length at some point, and your angles to units of degrees.  Set the ``length_conversion`` and ``angle_conversion`` values appropriately.

In [ ]:
center = 48000
length_conversion = 5000 # converts to cm
angle_conversion = 25600/360 #25600 motor pulses complete one full rotation
data['xcenter'] = (data['x'] - center)
data['xconverted'] = data['xcenter']/length_conversion
data['thetaconverted'] = data['theta']/angle_conversion
display(data)

In [ ]:
data_by_angle = []
fig,ax = plt.subplots()
for a in angles:
    x_val = data[data['theta']==a]['xconverted'] 
    rate_val = data[data['theta']==a]['rate']
    ax.plot(x_val,rate_val,label=a,marker='.')
ax.set_xlabel("Displacement(cm)")
ax.legend()

Well, now that we have the dimensions figured out, let's convert this series of 1d plots into a 2d plot of position versus angle.

To do this, we'll use the ``pivot_table`` function to select out the coincidence counts in terms of the converted $\theta$ and $x$ variables.  

If you want a bit of an explanation of what this does, see the [pivot table wikipedia page](https://en.wikipedia.org/wiki/Pivot_table#Mechanics).

In [ ]:
fig,ax = plt.subplots()
sinogram = data.pivot_table('rate', 'thetaconverted', 'xconverted', fill_value=0).values
ax.imshow(sinogram,aspect="auto")
# The rest of this gets a list of the angles & positions we're using so that we can scale the plot appropriately.
positions = data.pivot_table('rate', 'thetaconverted', 'xconverted', fill_value=0).keys() 
angle_labels =  data.pivot_table('rate', 'xconverted','thetaconverted', fill_value=0).keys()
ax.set_xticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ax.set_yticks(np.arange(0,len(sinogram[:,0]))[::2],angle_labels[::2]);

Now, let's look how this plot corresponds to our previous ones.  After shamelessly stealing from one of the [Matplotlib Tutorials](https://matplotlib.org/stable/tutorials/intermediate/arranging_axes.html), we can set up our sinogram plot side-by-side with some of the individual scans.  If how the two correlate isn't clear, talk with your TA or the lab staff to make sure you understand what's going on.

In [ ]:
fig = plt.figure(figsize = (9,6) ,constrained_layout=True)
spec = fig.add_gridspec(3, 3)

ax = fig.add_subplot(spec[:,:2])
sinogram = data.pivot_table('rate', 'thetaconverted', 'xconverted', fill_value=0).values
ax.set_title("Sinogram")
ax.imshow(sinogram,aspect="auto")
positions = data.pivot_table('rate', 'thetaconverted', 'xconverted', fill_value=0).keys() 
angles =  data.pivot_table('rate', 'xconverted','thetaconverted', fill_value=0).keys()
ax.set_xticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ax.set_yticks(np.arange(0,len(sinogram[:,0]))[::2],angles[::2]);

ax0 = fig.add_subplot(spec[0, 2])
ax0.plot(data[data['thetaconverted']==angles[0]]['xconverted'], sinogram[0])
ax0.set_title("0 degree scan")
ax.axhline(0)
axm = fig.add_subplot(spec[1, 2])
axm.plot(data[data['thetaconverted']==angles[len(angles)//2]]['xconverted'], sinogram[len(angles)//2])
axm.set_title("{} degree scan".format(angles[len(angles)//2]))
ax.axhline(len(angles)//2,color='purple')
axf = fig.add_subplot(spec[2, 2])
axf.plot(data[data['thetaconverted']==angles[-1]]['xconverted'], sinogram[-1])
axf.set_title("{} degree scan".format(angles[-1]))
ax.axhline(len(angles)-1,color='red');

Now we have an inverse problem:  We know what the intensity of our source looks like as a function of distance and angle, but we want to know it in terms of x and y position in platform coordinates.

There are a number of ways we can go about doing this, but we'll start with the simplest first: projecting out our slices along the line of response for the detectors, rotating these projections by their associated angles, and then combining all these together to get a composite plot.

To project out out slices, we'll use `np.outer` to do an outer product; this turns our 1d slices into 2d matricies.

The code below does this for each slice in our data, and plots the results for the first and second slices.

In [ ]:
from scipy import ndimage
expanded_data = []
for item in sinogram:
    expand_vec = np.ones(item.size)  # this is used to get us a square matrix
    expanded_data.append(np.outer(expand_vec,item))
    
fig, (ax,ay) = plt.subplots(1,2,figsize= (8,4))
ax.imshow(expanded_data[0])
ax.set_xticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ax.set_yticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ay.imshow(expanded_data[-1])
ay.set_xticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ay.set_yticks(np.arange(0,len(sinogram[0]))[::2],positions[::2]);

One more issue we get into here: we'll want to rotate our data to overlay the scans, but first we need to be sure that we're rotating around the platform's axis (i.e. $x = 0$).  If we don't do this, then our reconstruction will smear everything out more and more the further off-axis we are.

To do this, we find the smallest positive `x` value in each slice and subtract it off of all the position values.

In [ ]:
fig,ax = plt.subplots()

a=0
dshift = data.query('theta == {}'.format(a))['xconverted']
shift_value = dshift[len(dshift)//2]

ax.imshow(expanded_data[0])
ax.set_xticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ax.set_yticks(np.arange(0,len(sinogram[0]))[::2],positions[::2]);
ax.set_title("Original")

fig, ax = plt.subplots()
ax.imshow(ndimage.shift(expanded_data[0],(0,shift_value)))
ax.set_xticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ax.set_yticks(np.arange(0,len(sinogram[0]))[::2],positions[::2]);
ax.set_title("Shifted")

shifted_data = np.empty_like(expanded_data)
for index,item in enumerate(expanded_data):
    shifted_data[index] = ndimage.shift(item,(0,shift_value))


Okay, we've made 2d representations of each slice, and we've figured out how to properly add an offset to the axes.  Now how do we rotate them?  Instead of writing code to do matrix rotation ourselves, we'll use scipy's `ndimage` library.  Unless you manually want to write a matrix rotation algorithm.  I'm not your boss, knock yourself out.

In [ ]:
rotated_data = []
for index, item in enumerate(shifted_data):
    rotated_data.append(ndimage.rotate(item,angles[index],reshape=False)) # we want Numpy to keep our square, 2d matricies.
    
fig, (ax,ay) = plt.subplots(2)
fig.tight_layout()
ax.imshow(rotated_data[0])
ax.set_title("First slice")
ay.imshow(rotated_data[1]) 
ay.set_title("Second slice")
ax.set_xticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ax.set_yticks(np.arange(0,len(sinogram[0]))[::2],positions[::2]);
ay.set_xticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ay.set_yticks(np.arange(0,len(sinogram[0]))[::2],positions[::2]);

The rotated part has some bad aliasing going on, but it more-or-less captures the results of our second scan.  We'll show how to reduce that a bit later, but for now let's start recombining data!

To do this, we'll make a composite image by adding together the coincidence counts for each of these expanded & rotated slices.  Locations where we don't see many coincidences at any angle will end up with low counts, and locations with lots of coincidences at similar angles will end up with high counts.  It may be easier to see what this looks like, so let's start by recombining just the first and third slices.

We'll also normalize the data by dividing by `n`, where `n` is the number of slices we're using.

----
For starters, let's add the first and last rotations together

In [ ]:
composite = rotated_data[0] + rotated_data[-1]
normalized = composite/2

fig, ax = plt.subplots()
ax.imshow(normalized)

If you have more than one source, you might notice that there are multiple areas corresponding to high intensities here.  That's because there are cross-terms where one source's maximum lines up with a different source.  Thankfully, this effect can be mitigated by adding in more rotations, which will supress the 'ghost' peaks. Let's do that now.

In [ ]:
composite = np.ones_like(rotated_data[0]) # We need a matrix full of 1s to start out with, we can't multiply an empty matrix with anything.
for item in rotated_data:
    composite += item
normalized = composite/len(rotated_data) #

fig, ax = plt.subplots()
ax.imshow(normalized)
ax.set_xticks(np.arange(0,len(sinogram[0]))[::2],positions[::2])
ax.set_yticks(np.arange(0,len(sinogram[0]))[::2],positions[::2]);

While likely quite blurry, you should have something that now at least vaguely resembles a few point sources.  Congratulations, you've now got the basics down!

# Alternate Plot Types
----
We can also make contour plots, showing regions of equal intensity.

In [ ]:
fig, ax = plt.subplots()

num_lines = 10
flipped = normalized[::-1] # This is Python sorcery that flips the data vertically so that it matches the other plots.
con=ax.contour(normalized,levels=num_lines)
ax.set_aspect('equal')
ax.clabel(con, inline=True, fontsize=8) 

Another useful option is to make a 3d projection of the sinogram

In [ ]:
positions 
angle_labels
P,A = np.meshgrid(positions,angle_labels)
fig = plt.figure()
ax = plt.axes(projection='3d')
truncated = np.where(sinogram>10,sinogram,np.zeros_like(sinogram)) # This takes out areas where the counts are less than 10
ax.contour3D(P, A, truncated ,50)
ax.set_xlabel('position')
ax.set_ylabel('angle')
ax.set_zlabel('coincidences');

Or, we can make a 3d plot of the final reconstruction

In [ ]:
positions 
angle_labels
X, Y= np.meshgrid(positions,positions)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X,Y,normalized ,50)
ax.set_xlabel('x position')
ax.set_ylabel('y position')
ax.set_zlabel('coincidences');

We can also do this for each of our slices, giving a 3d view of how our reconstruction step-by-step.

# Advanced Techniques
----

We can also use a professional reconstruction technique: a [radon transform](https://en.wikipedia.org/wiki/Radon_transform).  At heart this technique uses a Fourier transform of slices, recombines these in a polar format, and then does an inverse Fourier transform.  You can try and work this out for yourself, but we can also use a package from `skimage` to do this for us with our data.  Since we're starting with slice/angle data, we'll be doing an inverse transformation.  See [this link](https://scikit-image.org/docs/stable/auto_examples/transform/plot_radon_transform.html) for more on how the `iradon` function works.

You'll need to install the `skimage` package in Anaconda for this to work.

The following shows what the inverse radon transform does, and then shows the difference between it and our reconstruction.  The results are normalized to a maximum coincidence rate of `1` to make the shapes comparable.

In [ ]:

from skimage.transform import iradon
# the possible filters are 'ramp', 'shepp-logan', 'cosine', 'hamming', and 'hann'
reconstruction = iradon(sinogram.T,theta=angles,filter_name='cosine')
# the transpose makes it line up with our other method
fig, ax = plt.subplots()
ax.imshow(reconstruction/np.max(reconstruction))
ax.set_title("Inverse radon transform data");
fig,ax = plt.subplots()
ax.imshow(normalized/np.max(normalized))
ax.set_title("Algebraic Reconstruction")
fig, ax = plt.subplots()
ax.imshow(reconstruction/np.max(reconstruction)-normalized/np.max(normalized))
ax.set_title("Difference")

Now this can be hard to interpret, so we'll check out what happens when we make a 3D plot.

In [ ]:
X, Y= np.meshgrid(positions,positions)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X,Y,reconstruction/np.max(reconstruction),50)
ax.set_xlabel('x position')
ax.set_ylabel('y position')
ax.set_zlabel('coincidences');
ax.set_title("Inverse Radon Transform")

X, Y= np.meshgrid(positions,positions)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X,Y,normalized/np.max(normalized),50)
ax.set_xlabel('x position')
ax.set_ylabel('y position')
ax.set_zlabel('coincidences');
ax.set_title("Algebraic Reconstruction")

X, Y= np.meshgrid(positions,positions)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X,Y,reconstruction/np.max(reconstruction)-normalized/np.max(normalized),50)
ax.set_xlabel('x position')
ax.set_ylabel('y position')
ax.set_zlabel('coincidences');
ax.set_title("Differences")